## Proyecto Music Stream


### Objetivo

1. Crear una bases de datos para almacenar información de canciones y artistas. 
2. Extraer información de la base de datos para responder preguntas sobre la música almacenada.

### Fases del proyecto 

**Fase 1: Extraer Datos**
1. Extraer información de Api Spotify
2. Extraer información de API Last FM
3. Guardar esa información en CSV

**Fase 2:Bases de datos**
1. Diseñar la Base de Datos
2. Crear la bases de datos
3. Insertar los valores en la bases de datos
   
**Fase 3:Extraer información**
1. Extraer información de la base de datos para responder preguntas sobre la música almacenada.
   

## Comencemos... Fase 1.1 Extraer información de Api Spotify

- Debemos usar la librería spotipy para extraer información de la API de Spotify: https://spotipy.readthedocs.io/en/2.24.0/ 
- Debemos hacer una cuenta en Spotify para obtener las credenciales necesarias para usar la API de Spotify.
- `!pip install spotipy`



In [3]:
!pip install spotipy

   ---------------------------------------- 0.0/261.5 kB ? eta -:--:--
   ------ --------------------------------- 41.0/261.5 kB 2.0 MB/s eta 0:00:01
   ------------------------------- -------- 204.8/261.5 kB 3.1 MB/s eta 0:00:01
   ---------------------------------------- 261.5/261.5 kB 2.7 MB/s eta 0:00:00


In [4]:
import spotipy

In [6]:
from spotipy.oauth2 import SpotifyClientCredentials

In [7]:
# Coloca tus credenciales aquí
CLIENT_ID = 'a018711755a146b2a678a93741dc6041'
CLIENT_SECRET = '51fa44e4b57343f19c810b9a75089da6'

# Autenticación
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

### Entendiendo el API de Spotipy 


- La documentación del API Spotify, el método que vamos a usar: https://spotipy.readthedocs.io/en/2.24.0/index.html#spotipy.client.Spotify.search


Tenemos disponible el método `search(q, limit=10, offset=0, type='track', market=None)` donde: 

- `search` busca un artículo que cumpla las condiciones de la búsqueda, que van ser: 
- q: la cadena de búsqueda limitando la búsqueda utilizando filtros de campo. Los filtros disponibles son `álbum`, `artista`, `pista`, `año`, `upc`, `etiqueta:hipster`, `etiqueta:nuevo`, isrc y `género`. Cada filtro de campo solo se aplica a ciertos tipos de resultados.
  - Los filtros de artista y año se pueden utilizar al buscar álbumes, artistas y pistas. Puede filtrar por un solo año o por un rango (por ejemplo, 1955-1960).
  - El filtro de álbumes se puede utilizar mientras se buscan álbumes y pistas
  - El filtro de género se puede utilizar al buscar artistas y pistas.
  - Los filtros isrc y track se pueden utilizar mientras se buscan pistas.
  - Los filtros upc, tag:new y tag:hipster solo se pueden utilizar durante la búsqueda de álbumes.
  -  El filtro etiqueta:nuevo devolverá álbumes lanzados en las últimas dos semanas y etiqueta:hipster se puede utilizar para devolver sólo álbumes con el 10% de popularidad más bajo. 

- limit: el número máximo de elementos que se devolverán. El valor predeterminado es 10 y el límite máximo es 50.
- tipo: Una lista separada por comas de tipos de elementos para buscar. Los resultados de la búsqueda incluyen visitas de todos los tipos de elementos especificados. Por ejemplo: q=abacab&type=album,track devuelve álbumes y pistas que coinciden con "abacab". Valores permitidos: "album", "artist", "playlist", "track", "show", "episode", "audiobook"


- Aquí podemos obtener datos como artista, genero, tipo (canción o albuum), nombre, año de lanzamiento, 

  


### Como usar el método de search de Spotipy

In [8]:
#Estamos buscando las canciones que sean de genero rock del 2020, la consulta lo ponemos en la query
datos = sp.search(q="genre:rock year:2020", type='track', limit=50, offset=0)
data_track_album = sp.search(q="genre:rock year:2020", type='track,album', limit=50, offset=0)#para ambos

In [9]:
#aqui vemos que me trae como respuesta, y vemos en el resultado qeu viene dentro de tracks
datos.keys()

dict_keys(['tracks'])

In [ ]:
datos

In [ ]:

# Aqui veo la info que viene en tracks datos[`tracks`], pero no es suficiente, al parecer la información viene en items, entonces hago un datos['tracks']['items'], como me sigue sin ver bien solo voy a ver un elemento a ver como se ve y hago datos['tracks']['items'][0]
datos['tracks']['items'][0]

In [13]:

track_only_one = datos['tracks']['items'][2]

In [ ]:
track_only_one

In [34]:

track_object = {
    'name_track': track_only_one['name'],
    'name_artist': track_only_one['artists'][0]['name'],
    'genre': "rock", #no lo devuelve, lo tengo que poner según el género que esté buscando
    'type': track_only_one['type'], 
    'year': track_only_one['album']['release_date'][:4] 
}
track_object

## ahora esto solo está hecho para una canción, de todo el resultado que me devuelve
#tendremos que hacer esto mismo para todo el listado: pista hay que hacer un for
# la información la guardaremos en un listado de objectos, donde cada objetos es la información de la canción

resultados_totales = []
for item in datos['tracks']['items']:
    resultados_totales.append({
             'name_track': item['name'],
            'name_artist': item['artists'][0]['name'],
            'genre': "rock", #no lo devuelve, lo tengo que poner según el género que esté buscando
            'type': item['type'], 
            'year': item['album']['release_date'][:4] })

In [ ]:
#para ver mi listado de canciones
resultados_totales

### Ahora vamos a hacerlo para nuestro problema..... ahora si!  🚀

In [45]:
# Elegimos los géneros
genres = ['pop', 'rock', 'hip-hop', 'electronic']
anio_inicio = 2002
anio_final = 2002

results = []
artistas = []

In [46]:
#Tendremos que iterar en los géneros y realizarla búsqueda de cada género y rango de año, ejemplo:
for genre in genres:
    results_genre = sp.search(q=f'genre:{genre} year:{anio_inicio}-{anio_final}', type='track,album', limit=50)
    for item in results_genre['tracks']['items']: ## ejemplo solo para canciones
        artistas.append(item['artists'][0]['name']) ## añado a la lista de artistas
        results.append({
                'name_track': item['name'],
                'name_artist': item['artists'][0]['name'],
                'genre': "rock", #no lo devuelve, lo tengo que poner según el género que esté buscando
                'type': item['type'], 
                'year': item['album']['release_date'][:4] })
    for item in results_genre['albums']['items']: ## ejemplo solo para albumes
        artistas.append(item['artists'][0]['name']) ## añado a la lista de artistas
        results.append({
                'name_track': item['name'],
                'name_artist': item['artists'][0]['name'],
                'genre': "rock", #no lo devuelve, lo tengo que poner según el género que esté buscando
                'type': item['type'], 
                'year': item['release_date'][:4] })
len(results)
    # Parece que hay información importante en items, ¿no? y ya sabemos como la tenemos que sacar, tanto para álbumes como para tracks
    # Aquí tenemos que hacer lo mismo que anteriormente buscar que datos tenemos para cada caso, ya sea albumes o canciones 
    #Consejos: buscar un límite de elementos, por ejemplo 500 por álbumes y canciones, una vez que tengáis el código cada una de vosotras se pone con un añi 
    #¡Ánimo!

Extrayendo datos para el género: pop
Extrayendo datos para el género: rock
Extrayendo datos para el género: hip-hop
Extrayendo datos para el género: electronic


201

## Guardar toda la info en un csv

In [ ]:
# Esto me sirve simplemente para ver qeu informacion pude extraer de la API de Spotify.
# Convertimos la lista de diccionarios a un DataFrame
df = pd.DataFrame(results)

# Guardamos el DataFrame en un archivo CSV
df.to_csv('spotify_data_final.csv', index=False, encoding='utf-8')

## Voy a ir poco más y sacar toda la info de los artistas, al menos el nombre y lo completo con la consulta a lastfm

In [ ]:
import pandas as pd
unique_artists = pd.Series(artistas).unique()
unique_artists

## Proyecto Music Stream

## Seguimos... Fase 1.2 Extraer información de Api Last FM

- Hay que ir a la página de Last FM y registrarse para obtener una API KEY. https://www.last.fm/api 
- Consultar la documentación de la API de Last FM para saber como hacer las consultas.https://www.last.fm/api 
- La dirección URL de la API de Last FM es: http://ws.audioscrobbler.com/2.0/
- Aquí podemos obtener información por artista, como: biografia, oyentes, numero de reproducciones, artistas similares


In [ ]:
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
#pip install mysql-connector -> en caso de no tenerla
import mysql.connector
from mysql.connector import errorcode

# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
#pip install requests-> en caso de no tenerla
#pip install pandas-> en caso de no tenerla
#pip install numpy-> en caso de no tenerla
import requests
import pandas as pd

# conexion a spotify
# -----------------------------------------------------------------------
#pip install spotipy -> en caso de no tenerla
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [15]:
# Este script consulta los artistas en base a una lista que yo le paso (que traigo desde la consulta de spotify). Su informacion sera artista, 
# Biography, Listeners, Playcount, Similar Artists


# API Key obtenida de last.fm
API_KEY = 'd97c59010214fb89dd7e101407646224'  # Reemplaza con tu API Key válida de Last.fm.
BASE_URL = 'http://ws.audioscrobbler.com/2.0/' # esta base siempre es la misma.

In [ ]:
# Lista para almacenar los datos que luego guardaremos en un DataFrame
artist_data = []
params_info = {
        'method': 'artist.getinfo',
        'artist': "Madonna",
        'api_key': API_KEY,
        'format': 'json'
    }
response_info = requests.get(BASE_URL, params=params_info)
data_info = response_info.json()
data_info


In [ ]:
## Vamos a crear un objeto artista 

artista = {
    'Artist': "Madonna",
    'Biography': data_info['artist'].get('bio', {}).get('summary'),
    'Listeners': data_info['artist'].get('stats', {}).get('listeners', "Sin datos"),
    'Playcount': data_info['artist'].get('stats', {}).get('playcount', "Sin datos")
}
artista


In [ ]:
## Para obtener los artistas similares de un artisa
params_similar = {
        'method': 'artist.getsimilar',
        'artist': "Madonna",
        'api_key': API_KEY,
        'format': 'json'
    }

response_similar = requests.get(BASE_URL, params=params_similar)
data_similar = response_similar.json()
data_similar['similarartists']['artist']


In [ ]:
# Hago un listado con los valores similares
similar_artists=[]
for artist in data_similar['similarartists']['artist']:
    similar_artists.append(artist['name'])
similar_artists


In [ ]:
artista['similar_artists'] = similar_artists
artista

### Ejemplo de resultado:

- Una tabla con el nombre de artista, reproducciones, oyentes, biografia y artistas similares
- Una tabla de canciones: con el nombre de artista, nombre de cancion, nombre del album, año, genero, id y popularidad
